# Model Training
The following code blocks were used for training the triplet loss model. 

### Imports

In [3]:

# Got help from multiple web sources, notably:
# https://stackoverflow.com/questions/47727679/triplet-model-for-image-retrieval-from-the-keras-pretrained-network
# https://ksaluja15.github.io/Learning-Rate-Multipliers-in-Keras/
# https://keras.io/preprocessing/image/
# https://github.com/keras-team/keras/issues/3386
# https://github.com/keras-team/keras/issues/8130
# https://github.com/noelcodella/tripletloss-keras-tensorflow



# GLOBAL DEFINES
WIDTH = 96
HEIGHT = 96
CHANNELS = 3
SEED = 1337

import sys
import ssl 
ssl._create_default_https_context = ssl._create_unverified_context
import numpy as np
import matplotlib.pyplot as plt
import cv2
np.random.seed(SEED)

# TensorFlow Includes
import tensorflow as tf
#from tensorflow.contrib.losses import metric_learning
tf.random.set_seed(SEED)

# Keras Imports & Defines 
import keras
import keras.applications
from keras import backend as K
from keras.models import Model
from keras import optimizers
import keras.layers as kl
from keras.layers import Input, Layer
from keras.preprocessing.image import ImageDataGenerator
from model import create_model


from keras.legacy import interfaces
import keras.backend as K
from keras.optimizers import Optimizer

import keras as tf
print(tf.__version__)

# Generator object for data augmentation.
# Can change values here to affect augmentation style.
datagen = ImageDataGenerator(  rotation_range=90,
                                width_shift_range=0.05,
                                height_shift_range=0.05,
                                zoom_range=0.1,
                                horizontal_flip=True,
                                vertical_flip=True,
                                )


Using TensorFlow backend.


2.3.1


### Optimiser
 As hilighted in the report, a SGD optimiser was used to update the learning rates for each layer.

In [4]:

class LR_SGD(Optimizer):
    """Stochastic gradient descent optimizer.

    Includes support for momentum,
    learning rate decay, and Nesterov momentum.

    # Arguments
        lr: float >= 0. Learning rate.
        momentum: float >= 0. Parameter updates momentum.
        decay: float >= 0. Learning rate decay over each update.
        nesterov: boolean. Whether to apply Nesterov momentum.
    """

    def __init__(self, learning_rate=0.01, momentum=0., decay=0.,
                 nesterov=False,multipliers=None,**kwargs):
        super(LR_SGD, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.learning_rate = K.variable(learning_rate, name='learning_rate')
            self.momentum = K.variable(momentum, name='momentum')
            self.decay = K.variable(decay, name='decay')
        self.initial_decay = decay
        self.nesterov = nesterov
        self.lr_multipliers = multipliers

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        learning_rate = self.learning_rate
        if self.initial_decay > 0:
            learning_rate *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))
        # momentum
        shapes = [K.int_shape(p) for p in params]
        moments = [K.zeros(shape) for shape in shapes]
        self.weights = [self.iterations] + moments
        for p, g, m in zip(params, grads, moments):
            
            matched_layer = [x for x in self.lr_multipliers.keys() if x in p.name]
            if matched_layer:
                new_lr = learning_rate * self.lr_multipliers[matched_layer[0]]
            else:
                new_lr = learning_rate

            v = self.momentum * m - new_lr * g  # velocity
            self.updates.append(K.update(m, v))

            if self.nesterov:
                new_p = p + self.momentum * v - new_lr * g
            else:
                new_p = p + v

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'learning_rate': float(K.get_value(self.learning_rate)),
                  'momentum': float(K.get_value(self.momentum)),
                  'decay': float(K.get_value(self.decay)),
                  'nesterov': self.nesterov}
        base_config = super(LR_SGD, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

### Generator
Generator function for data augmentation

In [5]:
def createDataGen(X1, X2, X3, Y, b):

    local_seed = T_G_SEED
    genX1 = datagen.flow(X1,Y, batch_size=b, seed=local_seed, shuffle=False)
    genX2 = datagen.flow(X2,Y, batch_size=b, seed=local_seed, shuffle=False)
    genX3 = datagen.flow(X3,Y, batch_size=b, seed=local_seed, shuffle=False)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            X3i = genX3.next()

            yield [X1i[0], X2i[0], X3i[0]], X1i[1]

### Create model
This function creates the model to be trained

In [13]:
def createModel(emb_size):

    # Initialize a ResNet50_ImageNet Model
    resnet_input = kl.Input(shape=(WIDTH,HEIGHT,CHANNELS))
    resnet_model = keras.applications.resnet50.ResNet50(weights='imagenet', include_top = False, input_tensor=resnet_input)

    # New Layers over ResNet50
    net = resnet_model.output
    net = kl.GlobalAveragePooling2D(name='gap')(net)
    net = kl.Dense(emb_size,activation='relu',name='t_emb_1')(net)
    net = kl.Lambda(lambda  x: K.l2_normalize(x,axis=1), name='t_emb_1_l2norm')(net)

    # model creation
    base_model = Model(resnet_model.input, net, name="base_model")

    # triplet framework, shared weights
    input_shape=(WIDTH,HEIGHT,CHANNELS)
    input_anchor = kl.Input(shape=input_shape, name='input_anchor')
    input_positive = kl.Input(shape=input_shape, name='input_pos')
    input_negative = kl.Input(shape=input_shape, name='input_neg')

    net_anchor = base_model(input_anchor)
    net_positive = base_model(input_positive)
    net_negative = base_model(input_negative)

    # The Lamda layer produces output using given function. Here its Euclidean distance.
    positive_dist = kl.Lambda(euclidean_distance, name='pos_dist')([net_anchor, net_positive])
    negative_dist = kl.Lambda(euclidean_distance, name='neg_dist')([net_anchor, net_negative])
    tertiary_dist = kl.Lambda(euclidean_distance, name='ter_dist')([net_positive, net_negative])

    # This lambda layer simply stacks outputs so both distances are available to the objective
    stacked_dists = kl.Lambda(lambda vects: K.stack(vects, axis=1), name='stacked_dists')([positive_dist, negative_dist, tertiary_dist])

    model = Model([input_anchor, input_positive, input_negative], stacked_dists, name='triple_siamese') 
    
    
    # Setting up optimizer designed for variable learning rate

    # Variable Learning Rate per Layers
    lr_mult_dict = {}
    last_layer = ''
    for layer in resnet_model.layers:
        # comment this out to refine earlier layers
        # layer.trainable = False  
        # print layer.name
        lr_mult_dict[layer.name] = 1
        # last_layer = layer.name
    lr_mult_dict['t_emb_1'] = 100

    base_lr = 0.1
    momentum = 0.9
    v_optimizer = LR_SGD(learning_rate=base_lr, momentum=momentum, decay=0.0, nesterov=False, multipliers = lr_mult_dict)
   
    model.compile(optimizer=v_optimizer, loss=triplet_loss, metrics=[accuracy])

    return model

### Triplet Loss

Implements triplet loss function

In [14]:
def triplet_loss(y_true, y_pred):
    margin = K.constant(1)
    return K.mean(K.maximum(K.constant(0), K.square(y_pred[:,0,0]) - 0.5*(K.square(y_pred[:,1,0])+K.square(y_pred[:,2,0])) + margin))

def accuracy(y_true, y_pred):
    return K.mean(y_pred[:,0,0] < y_pred[:,1,0])

def l2Norm(x):
    return  K.l2_normalize(x, axis=-1)

def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))


In [20]:
# loads an image and preprocesses
def read_image(loc):
    t_image = cv2.imread(loc)
    t_image = cv2.resize(t_image, (HEIGHT,WIDTH))
    t_image = t_image.astype("float32")
    t_image = keras.applications.resnet50.preprocess_input(t_image, data_format='channels_last')

    return t_image

### Read image list
The function below loads a set of images from a text index file

In [21]:
def read_image_list(flist, start, length):

    with open(flist) as f:
        content = f.readlines() 
    #print(flist)
    #print(content)
    content = [x.strip().split()[0] for x in content] 

    datalen = length
    if (datalen < 0):
        datalen = len(content)

    if (start + datalen > len(content)):
        datalen = len(content) - start
 
    imgset = np.zeros((datalen, HEIGHT, WIDTH, CHANNELS))

    for i in range(start, start+datalen):
        if ((i-start) < len(content)):
            imgset[i-start] = read_image(content[i])

    return imgset


### Extract
Extracts model weights, for a given set of images and writes them to a textfile. Also produces a single array list input layer

In [22]:
def extract(argv):

    if len(argv) < 3:
        print ("Usage: \n\t <Model Prefix> <Input Image List (TXT)> <Output File (TXT)> \n\t\tExtracts triplet-loss model")
        return

    modelpref = argv[0]
    imglist = argv[1]
    outfile = argv[2]

    with open(modelpref + '.json', "r") as json_file:
        model_json = json_file.read()

    loaded_model = keras.models.model_from_json(model_json)
    loaded_model.load_weights(modelpref + '.h5')

    base_model = loaded_model.get_layer('base_model')

    # create a new single input
    input_shape=(WIDTH,HEIGHT,CHANNELS)
    input_single = kl.Input(shape=input_shape, name='input_single')
    
    # create a new model without the triple loss
    net_single = base_model(input_single)
    model = Model(input_single, net_single, name='embedding_net')
    model.save(outfile + '.h5')
    
    chunksize = 1000
    total_img = file_numlines(imglist)
    total_img_ch = int(np.ceil(total_img / float(chunksize)))

    with open(outfile, 'w') as f_handle:

        for i in range(0, total_img_ch):
            imgs = read_image_list(imglist, i*chunksize, chunksize)

            vals = model.predict(imgs)
    
            np.savetxt(f_handle, vals)


    return



### Learn

Calls createModel to create the model and then trains it using data in locations specified by user

In [23]:
def learn(argv):
    
    if len(argv) < 10:
        print ("Usage: \n\t <Train Anchors (TXT)> <Train Positives (TXT)> <Train Negatives (TXT)> <Val Anchors (TXT)> <Val Positives (TXT)> <Val Negatives (TXT)> <embedding size> <batch size> <num epochs> <output model> \n\t\tLearns triplet-loss model")
        return

    in_t_a = argv[0]
    in_t_b = argv[1]
    in_t_c = argv[2]

    in_v_a = argv[3]
    in_v_b = argv[4]
    in_v_c = argv[5]

    emb_size = int(argv[6])
    batch = int(argv[7])
    numepochs = int(argv[8])
    outpath = argv[9] 

    # chunksize is the number of images we load from disk at a time
    chunksize = batch*100
    total_t = file_numlines(in_t_a)
    total_v = file_numlines(in_v_b)
    total_t_ch = int(np.ceil(total_t / float(chunksize)))
    total_v_ch = int(np.ceil(total_v / float(chunksize)))

    print ("Dataset has " + str(total_t) + " training triplets, and " + str(total_v) + " validation triplets.")

    print ("Creating a model ...")
    model = createModel(emb_size)

    print ("Training loop ...")
    
    # manual loop over epochs to support very large sets of triplets
    for e in range(0, numepochs):

        for t in range(0, total_t_ch):

            print ('Epoch ' + str(e) + ': train chunk ' + str(t+1) + '/ ' + str(total_t_ch) + ' ...')

            print ('Reading image lists ...')
            anchors_t = read_image_list(in_t_a, t*chunksize, chunksize)
            positives_t = read_image_list(in_t_b, t*chunksize, chunksize)
            negatives_t = read_image_list(in_t_c, t*chunksize, chunksize)
            Y_train = np.random.randint(2, size=(1,2,anchors_t.shape[0])).T
            
            print ('Starting to fit ...')
            # This method does NOT use data augmentation
            model.fit([anchors_t, positives_t, negatives_t], Y_train, epochs=numepochs,  batch_size=batch)
            
            # This method uses data augmentation
            #model.fit_generator(generator=createDataGen(anchors_t,positives_t,negatives_t,Y_train,batch), steps_per_epoch=len(Y_train) / batch, epochs=1, shuffle=False, use_multiprocessing=True)
        
        # In case the validation images don't fit in memory, we load chunks from disk again. 
        val_res = [0.0, 0.0]
        total_w = 0.0
        for v in range(0, total_v_ch):

            print ('Loading validation image lists ...')
            print ('Epoch ' + str(e) + ': val chunk ' + str(v+1) + '/ ' + str(total_v_ch) + ' ...')
            anchors_v = read_image_list(in_v_a, v*chunksize, chunksize)
            positives_v = read_image_list(in_v_b, v*chunksize, chunksize)
            negatives_v = read_image_list(in_v_c, v*chunksize, chunksize)
            Y_val = np.random.randint(2, size=(1,2,anchors_v.shape[0])).T

            
            # Weight of current validation measurement. 
            # if loaded expected number of items, this will be 1.0, otherwise < 1.0, and > 0.0.
            w = float(anchors_v.shape[0]) / float(chunksize)
            total_w = total_w + w

            curval = model.evaluate([anchors_v, positives_v, negatives_v], Y_val, batch_size=batch)
            val_res[0] = val_res[0] + w*curval[0]
            val_res[1] = val_res[1] + w*curval[1]

        val_res = [x / total_w for x in val_res]

        print ('Validation Results: ' + str(val_res))

    print ('Saving model ...')

    # Save the model and weights
    model.save(outpath + '.h5')

    # Due to some remaining Keras bugs around loading custom optimizers
    # and objectives, we save the model architecture as well
    model_json = model.to_json()
    with open(outpath + '.json', "w") as json_file:
        json_file.write(model_json)

    return

def file_numlines(fn):
    with open(fn) as f:
        return sum(1 for _ in f)

### Driver

This is the 'driver cell'. 
Arguments are passed into the argv array. A format for passing the arguments can be seen by running the cell without assigning any values to the argv array.

In [1]:
    
argv = []
if len(argv) < 2:
    print ('Usage: \n\t -learn <Train Anchors (TXT)> <Train Positives (TXT)> <Train Negatives (TXT)> <Val Anchors (TXT)> <Val Positives (TXT)> <Val Negatives (TXT)> <embedding size> <batch size> <num epochs> <output model prefix> \n\t -extract <Model Prefix> <Input Image List (TXT)> <Output File (TXT)> \n\t\tBuilds and scores a triplet-loss model ')
if(len(argv) > 0):
    if 'learn' in argv[0]:
        learn(argv[1:])
    elif 'extract' in argv[0]:
        extract(argv[1:])    


Usage: 
	 -learn <Train Anchors (TXT)> <Train Positives (TXT)> <Train Negatives (TXT)> <Val Anchors (TXT)> <Val Positives (TXT)> <Val Negatives (TXT)> <embedding size> <batch size> <num epochs> <output model prefix> 
	 -extract <Model Prefix> <Input Image List (TXT)> <Output File (TXT)> 
		Builds and scores a triplet-loss model 


# Preparing the dataset

The following blocks of code were used to prepare the dataset and also generate graphs for model training interpretations

In [ ]:
'''Copies all images from all folders in a directory into one folder'''
import os
from os.path import join as pjoin
from skimage import io
img_dir = 'dataset'
def load_data(data_dir): #lfw is the originalPath

    count = 0
    print("started...")
    for guys in os.listdir(data_dir):
        person_dir = pjoin(data_dir, guys)
        for i in os.listdir(person_dir):
            image_dir = pjoin(person_dir, i)
            img22 = io.imread(image_dir, pilmode='RGB')
            io.imsave(pjoin(img_dir, i), img22)
            count = count+1
    print("Copied: " + str(count)
    return count

sourcepath = 'lfw'
load_data(sourcepath)

Method used in the naming of image files in a predefined format to make reading images from index file easy


In [ ]:
from os.path import join as pjoin
def imagedir(basename):
    name = basename[0] + "_" 
    if(len(basename[1]) == 1):
        name = name + "000" + basename[1]
    elif(len(basename[1]) == 2):
        name = name + "00" + basename[1]
    elif(len(basename[1]) == 3):
        name = name + "0" + basename[1]
    else:
        name = name + basename[1]
    data_dir = "dataset"
    name = name + ".jpg\n"
    name = pjoin(data_dir, name)
    return name

print("starting...")       
with open('val_individuals.txt') as f1:
    with open('val_pairs.txt') as f2:
        with open('val_anchor.txt', 'a') as f3:
            with open('val_positive.txt', 'a') as f4:
                with open('val_negative.txt', 'a') as f5:
                    pair_lines = f2.readlines()
                    positive = ""
                    negative = ""
                    anchor = ""
                    count = 0
                    for line in f1:
                        words_ind = line.split()
                        
                        for pair in pair_lines:
                            names = pair.split()
                            if(words_ind[0] == names[0]):
                                count = count + 1
                                pos = imagedir(names[:2])
                                neg = imagedir(names[2:])
                                anchor = imagedir(words_ind)
                                f3.write(anchor)
                                f4.write(pos)
                                f5.write(neg)
                                #print(anchor)
                            elif ( names[2] == words_ind[0] ):
                                count = count + 1
                                pos = imagedir(names[2:])
                                neg = imagedir(names[:2])
                                anchor = imagedir(words_ind)
                                f3.write(anchor)
                                f4.write(pos)
                                f5.write(neg)
                    print(str(count) + " records written to file")
                        
                        
                                
                    
print("done")


This partitions the dataset into triplets: anchor image, positive and negative images. These are then written to three defferent index files.


In [ ]:

print("starting...")       
with open('anchor.txt', 'r') as f1:
    with open('positive.txt', 'r') as f2:
        with open('negative.txt', 'r') as f3:
            with open("a.txt", "w") as fa:
                with open("b.txt", "w") as fb:
                    with open ("c.txt", "w") as fc:
                        count = 0
                        for (a, b, c) in zip(f1, f2, f3):
                             if(a.strip("\n") != b.strip("\n")):
                                    fa.write(a)
                                    fb.write(b)
                                    fc.write(c)
                                    count = count + 1
                  
                #if (count > 1):
                 #   print(words_ind[0]+" "+str(count))



print("done "+ str(count))